In [2]:
import pandas as pd
import pickle
from helper_functions import Helper
from helper_functions import CoinAPI
import requests

In [ ]:
# TODO START THE BELOW PULL AT 10 AM ON TUESDAY

In [ ]:
if __name__ == "__main__":
    # import api key
    API_KEY_FP = '../../admin/coinapi.txt'
    with open(API_KEY_FP) as f:
        API_KEY = f.readlines()[0].strip()

    # Set args
    CW_IN_FP = '../data/derived/cm_to_coinapi_cw.pkl'
    ASSET_IN_FP = '../data/clean/asset_universe_dict.pickle'
    BASE_URL   = 'https://rest.coinapi.io/v1/'
    BASE_HEADERS = {'X-CoinAPI-Key': API_KEY}
    LEGIT_US_EXCHANGES = ['BINANCEUS', 'BITSTAMP', 'COINBASE', 'CRYPTOCOM', 'FTXUS', 
        'GEMINI', 'KRAKEN', 'KUCOIN']
    TARGET_FREQ = '1HRS'
    TIME_START = '2016-07-01'
    TIME_END = '2023-01-02'
    PANEL_OUT_FP = '../data/raw/coinapi_panel_hourly.pkl'

    # confirm api is working
    url = 'https://www.coinapi.io/api/subscriptions/usage/rest/history'
    print(requests.get(url, headers=BASE_HEADERS).json())    
    
    # Import asset universe and cw
    cw_df = pd.read_pickle(CW_IN_FP)
    with open(ASSET_IN_FP, "rb") as f:
        asset_universe_dict = pickle.load(f)
    asset_universe_cm = Helper.findUniqueAssets(asset_universe_dict)
    asset_universe = list(cw_df[cw_df.asset_cm.isin(asset_universe_cm)].asset_coinapi.values)

    # pull relevant markets
    markets_df = CoinAPI.pullMarketInfo(BASE_URL, BASE_HEADERS, LEGIT_US_EXCHANGES, asset_universe)

    # pull coinapi aggregated prices
    prices_df = CoinAPI.pullExchangeRates(BASE_URL, BASE_HEADERS, ['USDC', 'USDT'] + asset_universe,
                    TARGET_FREQ, TIME_START, TIME_END)
    macro_df, prices_df = CoinAPI.seperateMacroAndAssetRefPrices(prices_df)

In [ ]:
# TODO TEMP SAVE OF MACRO AND PRICES
markets_df.to_pickle('temp_markets.pkl')
macro_df.to_pickle('temp_macro.pkl')
prices_df.to_pickle('temp_macro.pkl')

In [ ]:
# pull market prices and volumes
master_markets_list = list(markets_df.symbol_id.values)

In [ ]:
# confirm api is working
url = 'https://www.coinapi.io/api/subscriptions/usage/rest/history'
print(requests.get(url, headers=BASE_HEADERS).json())    


In [ ]:
len(master_markets_list)

In [ ]:
len(master_markets_list)/165

In [ ]:
len(master_markets_list)/6.5

In [ ]:
# TODO RIP ONE MARKET PULL TO SEE HOW MUCH IT COSTS. LIKE 400?

In [ ]:
# TODO DECIDE ON SPACING

In [ ]:
# TODO 
end = 100
markets_list = master_markets_list[:end] # TODO CHANGE TO SOME LIST
raw_panel_df = CoinAPI.pullMarketData(BASE_URL, BASE_HEADERS, markets_list, 
                                TARGET_FREQ, TIME_START, TIME_END)

In [ ]:
master_raw_panel_df = pd.DataFrame()
master_raw_panel_df = pd.concat((master_raw_panel_df, raw_panel_df))
master_raw_panel_df.to_pickle('temp_panel_thru_100.pkl')

In [ ]:
# TODO TUESDAY
start = 50
end = 200
markets_list = master_markets_list[start:end] 
raw_panel_df = CoinAPI.pullMarketData(BASE_URL, BASE_HEADERS, markets_list, 
                                TARGET_FREQ, TIME_START, TIME_END)
master_raw_panel_df = pd.concat((master_raw_panel_df, raw_panel_df))
master_raw_panel_df.to_pickle('temp_panel_thru_200.pkl')

In [ ]:
# TODO W
start = 200
end = 375
markets_list = master_markets_list[start:end] 
raw_panel_df = CoinAPI.pullMarketData(BASE_URL, BASE_HEADERS, markets_list, 
                                TARGET_FREQ, TIME_START, TIME_END)
master_raw_panel_df = pd.concat((master_raw_panel_df, raw_panel_df))
master_raw_panel_df.to_pickle('temp_panel_thru_375.pkl')

In [ ]:
# TODO Th
start = 375
end = 550
markets_list = master_markets_list[start:end] 
raw_panel_df = CoinAPI.pullMarketData(BASE_URL, BASE_HEADERS, markets_list, 
                                TARGET_FREQ, TIME_START, TIME_END)
master_raw_panel_df = pd.concat((master_raw_panel_df, raw_panel_df))
master_raw_panel_df.to_pickle('temp_panel_thru_550.pkl')

In [ ]:
# TODO F
start = 550
end = 725
markets_list = master_markets_list[start:end] 
raw_panel_df = CoinAPI.pullMarketData(BASE_URL, BASE_HEADERS, markets_list, 
                                TARGET_FREQ, TIME_START, TIME_END)
master_raw_panel_df = pd.concat((master_raw_panel_df, raw_panel_df))
master_raw_panel_df.to_pickle('temp_panel_thru_725.pkl')

In [ ]:
# TODO 
start = 725
end = 900
markets_list = master_markets_list[start:end] 
raw_panel_df = CoinAPI.pullMarketData(BASE_URL, BASE_HEADERS, markets_list, 
                                TARGET_FREQ, TIME_START, TIME_END)
master_raw_panel_df = pd.concat((master_raw_panel_df, raw_panel_df))
master_raw_panel_df.to_pickle('temp_panel_thru_900.pkl')

In [ ]:
# TODO
start = 900
end = 1075
markets_list = master_markets_list[start:end] 
raw_panel_df = CoinAPI.pullMarketData(BASE_URL, BASE_HEADERS, markets_list, 
                                TARGET_FREQ, TIME_START, TIME_END)
master_raw_panel_df = pd.concat((master_raw_panel_df, raw_panel_df))
master_raw_panel_df.to_pickle('temp_panel_thru_1075.pkl')

In [ ]:
# TODO 
start = 1075
end = 1250
markets_list = master_markets_list[start:end] 
raw_panel_df = CoinAPI.pullMarketData(BASE_URL, BASE_HEADERS, markets_list, 
                                TARGET_FREQ, TIME_START, TIME_END)
master_raw_panel_df = pd.concat((master_raw_panel_df, raw_panel_df))
master_raw_panel_df.to_pickle('temp_panel_thru_200.pkl')

In [ ]:
# TODO 
start = 1250
markets_list = master_markets_list[start:] 
raw_panel_df = CoinAPI.pullMarketData(BASE_URL, BASE_HEADERS, markets_list, 
                                TARGET_FREQ, TIME_START, TIME_END)
master_raw_panel_df = pd.concat((master_raw_panel_df, raw_panel_df))
master_raw_panel_df.to_pickle('temp_pane.pkl')

In [ ]:


# clean the panel
panel_df = CoinAPI.cleanPanel(raw_panel_df, prices_df, macro_df, TARGET_FREQ)
panel_df.to_pickle(PANEL_OUT_FP)